## Importing Packages ##

In [2]:
import os
import sys
import subprocess
import tempfile
import random
from itertools import compress
import os.path as op
import numpy as np
import skimage.io as sio
import pandas as pd
import importlib
import math
from math import sqrt
import matplotlib.pyplot as plt
from os import listdir, getcwd, chdir
from os.path import isfile, join
from sklearn import linear_model
from sklearn import svm
import diff_classifier
import diff_classifier.utils as ut
import diff_classifier.msd as msd
import diff_classifier.features as ft
import diff_classifier.heatmaps as hm
import diff_classifier.aws as aws

## Defining filename and importing video - will eventually convert to a for loop to loop through list of videos ##

In [3]:
tiffname = 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.tiff'

In [4]:
os.chdir('/Users/claudialozano/Dropbox/PycharmProjects/AD_nanoparticle/diff_classifier/notebooks/development/MPT_Data/{}/'.format(tiffname.replace('.tiff', '')))
test = sio.imread(tiffname)
oshape = test.shape
oshape

(651, 2044, 2048)

In [5]:
os.getcwd()

'/Users/claudialozano/Dropbox/PycharmProjects/AD_nanoparticle/diff_classifier/notebooks/development/MPT_Data/P10F_NT_4DIV_40nm_slice_1_cortex_vid_2'

## Splitting original 2048x2044 video into smaller videos - takes the video imported just above and splits it into 64 256x256 smaller videos - could potentially adjust this to split into 25 or 36 vids, but for now keeping at 64 (easier to track the small vids) ##

In [ ]:
test2 = np.zeros((oshape[0], 2048, 2048), dtype=test.dtype)
test2[0:oshape[0], 0:oshape[1], :] = test

new_image = np.zeros((oshape[0], 512, 512), dtype=test.dtype)
names = []

for row in range(4):
    for col in range(4):
        new_image = test2[:, row*512:(row+1)*512, col*512:(col+1)*512]
        current = tiffname.split('.tif')[0] + '_%s_%s.tif' % (row, col)
        sio.imsave(current, new_image)
        names.append(current)

## For now, tracking is done manually using the plugin TrackMate in ImageJ. The cells located below this can be used to rename the trajectory .csv files to the proper 'Traj_... .csv' format ##

In [ ]:
os.getcwd()

In [8]:
import glob
files = glob.glob('Traj*.csv')
print(files)

['Traj_1_2_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_3_1_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_3_3_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_1_0_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_1_3_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_3_0_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_3_2_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_1_1_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_2_2_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_0_1_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_0_3_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_2_0_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_2_3_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_0_0_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_0_2_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv', 'Traj_2_1_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv']


## Defining list of all small videos to be quantified (calculating msds and features) ##

In [9]:
filename = tiffname.replace('.tiff', '')
rows = 4
cols = 4
ires = (512, 512)
frames = 651
names = []
for i in range(0,rows):
    for j in range(0,cols):
        names.append('{}_{}_{}.tif'.format(filename, i, j))

In [10]:
merged.shape

NameError: name 'merged' is not defined

In [11]:
names


['P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_0_0.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_0_1.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_0_2.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_0_3.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_1_0.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_1_1.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_1_2.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_1_3.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_2_0.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_2_1.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_2_2.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_2_3.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_3_0.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_3_1.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_3_2.tif',
 'P10F_NT_4DIV_40nm_slice_1_cortex_vid_2_3_3.tif']

## Calculating MSDs and features for the videos that have been tracked. This uses the Traj .csv files and generates an msd and features .csv file for the videos being quantified. It uses the multiple small vid Traj .csv files to generate a single msd and features .csv file. Essentially, this runs the kn.assemble_msds() function ##

In [14]:
ft_file = 'features_{}.csv'.format(filename)
msd_file = 'msd_{}.csv'.format(filename)
toadd_file = 'toadd_{}.csv'.format(filename)

counter = 0
for name in names:
    try:
    
        row = int(name.split(filename)[1].split('.')[0].split('_')[1])
        col = int(name.split(filename)[1].split('.')[0].split('_')[2])
    
        traj_file = "Traj_{}_{}_{}.csv".format(row, col, filename)
        local_name = traj_file
        print(local_name)
    
        if counter == 0:
            to_add = pd.read_csv(local_name)
            to_add = to_add.iloc[3:]
            del to_add['LABEL']
            del to_add['POSITION_Z']
            del to_add['POSITION_T']
            del to_add['RADIUS']
            del to_add['VISIBILITY']
            del to_add['MANUAL_SPOT_COLOR']
            del to_add['MEDIAN_INTENSITY_CH1']
            del to_add['MIN_INTENSITY_CH1']
            del to_add['MAX_INTENSITY_CH1']
            del to_add['TOTAL_INTENSITY_CH1']
            del to_add['STD_INTENSITY_CH1']
            #del to_add['ESTIMATED_DIAMETER']
            del to_add['CONTRAST_CH1']
            to_add = to_add.rename(columns={'ID':'Spot_ID', 'TRACK_ID':'Track_ID','QUALITY':'Quality','FRAME':'Frame', 'POSITION_X':'X','POSITION_Y':'Y', 'MEAN_INTENSITY_CH1':'Mean_Intensity', 'SNR_CH1':'SN_Ratio'})
            to_add.sort_values(['Track_ID', 'Frame'], ascending=[1,1])
            to_add = to_add.astype('float64')

            partids = to_add.Track_ID.unique()
            counter = 0
            for partid in partids:
                to_add.loc[to_add.Track_ID == partid, 'Track_ID'] = counter
                counter = counter +1
            to_add['X'] = to_add['X'] + ires[0]*col
            to_add['Y'] = ires[1]-to_add['Y']+ires[1]*(rows-1-row)
            merged = msd.all_msds2(to_add, frames=frames)
            to_add.to_csv(toadd_file)
            merged.to_csv(msd_file)
            print("Finished with row 0 col 0")
            break
            

        else:
        
            if merged.shape[0] > 0:
                print("MERGED_SHAPE:", merged.shape)
                to_add = pd.read_csv(local_name)
                to_add = to_add.iloc[3:]
                del to_add['LABEL']
                del to_add['POSITION_Z']
                del to_add['POSITION_T']
                del to_add['RADIUS']
                del to_add['VISIBILITY']
                del to_add['MANUAL_SPOT_COLOR']
                del to_add['MEDIAN_INTENSITY_CH1']
                del to_add['MIN_INTENSITY_CH1']
                del to_add['MAX_INTENSITY_CH1']
                del to_add['TOTAL_INTENSITY_CH1']
                del to_add['STD_INTENSITY_CH1']
                #del to_add['ESTIMATED_DIAMETER']
                del to_add['CONTRAST_CH1']
                to_add = to_add.rename(columns={'ID':'Spot_ID', 'TRACK_ID':'Track_ID','QUALITY':'Quality','FRAME':'Frame', 'POSITION_X':'X','POSITION_Y':'Y', 'MEAN_INTENSITY_CH1':'Mean_Intensity', 'SNR_CH1':'SN_Ratio'})
                to_add.sort_values(['Track_ID', 'Frame'], ascending=[1,1])
                to_add = to_add.astype('float64')

                partids = to_add.Track_ID.unique()
                counter = 0
                for partid in partids:
                    to_add.loc[to_add.Track_ID == partid, 'Track_ID'] = counter
                    counter = counter +1
                to_add['X'] = to_add['X'] + ires[0]*col
                print(ires[0]*col)
                to_add['Y'] = ires[1] - to_add['Y'] + ires[1]*(rows-1-row)
                to_add['Track_ID'] = to_add['Track_ID'] + max(merged['Track_ID']) +1
    
            
            else:
                print("YOOOO GOT HERE")
                to_add = pd.read_csv(local_name)
                to_add = to_add.iloc[3:]
                del to_add['LABEL']
                del to_add['POSITION_Z']
                del to_add['POSITION_T']
                del to_add['RADIUS']
                del to_add['VISIBILITY']
                del to_add['MANUAL_SPOT_COLOR']
                del to_add['MEDIAN_INTENSITY_CH1']
                del to_add['MIN_INTENSITY_CH1']
                del to_add['MAX_INTENSITY_CH1']
                del to_add['TOTAL_INTENSITY_CH1']
                del to_add['STD_INTENSITY_CH1']
                #del to_add['ESTIMATED_DIAMETER']
                del to_add['CONTRAST_CH1']
                to_add = to_add.rename(columns={'ID':'Spot_ID', 'TRACK_ID':'Track_ID','QUALITY':'Quality','FRAME':'Frame', 'POSITION_X':'X','POSITION_Y':'Y', 'MEAN_INTENSITY_CH1':'Mean_Intensity', 'SNR_CH1':'SN_Ratio'})
                to_add.sort_values(['Track_ID', 'Frame'], ascending=[1,1])
                to_add = to_add.astype('float64')

                partids = to_add.Track_ID.unique()
                counter = 0
                for partid in partids:
                    to_add.loc[to_add.Track_ID == partid, 'Track_ID'] = counter
                    counter = counter +1
                to_add['X'] = to_add['X'] + ires[0]*col
                print(to_add['X'] + ires[0]*col)
                to_add['Y'] = ires[1] - to_add['Y'] + ires[1]*(rows-1-row)
                to_add['Track_ID'] = to_add['Track_ID']
            
            #print("THIS IS MERGED SHAPE")
            #print(merged.shape)
            merged = merged.append(msd.all_msds2(to_add, frames=frames))
            print('Done calculating MSDs for row {} and col {}'.format(row, col))
    
        counter = counter +1

        merged.to_csv(msd_file)
        
    except pd.errors.EmptyDataError:
        print('Found empty file : {}'.format(name))


#merged_ft = ft.calculate_features(merged)
#merged_ft.to_csv(ft_file)
print('OUT')

Traj_0_0_P10F_NT_4DIV_40nm_slice_1_cortex_vid_2.csv
Finished with row 0 col 0
OUT


In [ ]:
os.chdir('/Users/claudialozano/Dropbox/PycharmProjects/AD_nanoparticle/diff_classifier/notebooks/development/MPT_Data/{}/'.format(tiffname.replace('.tiff', '')))
prefix = filename
msds = 'msd_{}.csv'.format(prefix)
feat = 'features_{}.csv'.format(prefix)
print(feat)
print(msds)
hm.plot_trajectories(prefix, resolution=512, rows=4, cols=4, upload=False, figsize=(12, 12), bucket = 'mckenna.data')
print('Successfully generated trajectory plot for {}'.format(prefix))
geomean, geoSEM = hm.plot_individual_msds(prefix, x_range=2, y_range=10, umppx=0.07, fps=33.33, upload=False, bucket='mckenna.data')
print('Successfully generated csv files for {}'.format(prefix))

In [ ]:
org_feat = 'orig_features_{}.csv'.format(prefix)
hm.plot_heatmap(org_feat, resolution=512, rows=4, cols=4, upload=False, figsize=(12, 12), bucket = 'mckenna.data')

feat = 'features_{}.csv'.format(prefix)
hm.plot_heatmap(feat, resolution=512, rows=4, cols=4, upload=False, figsize=(12, 12), bucket = 'mckenna.data')